In [155]:
'''
Enter the Title of the movie
Then click on "CELL->Run All" 
'''
title ="Wonder Woman"

In [156]:
import re
import sys
from bs4 import BeautifulSoup
import requests
import time
import tweepy
from textblob import TextBlob
import pandas as pd
from tweepy import OAuthHandler
import pandas as pd
import json
import pickle
import csv
%matplotlib inline

# load the model from disk
model = pickle.load(open('finalized_model.sav', 'rb'))

In [157]:

with open('predict.csv', 'w') as csvfile:
    csvfile.truncate()
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Actor popularity','Director popularity','Sentiment','num_theaters','runtime_mins','releaseYear','CULT_INDEX','CPI','prod_budget_ADJ'])
    csvfile.close()
    
with open('predict1.csv', 'w') as csvfile:
    csvfile.truncate()
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['genre','distributor','Director','leadActor'])
    csvfile.close()

In [158]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = '34Z3k2QwDjvWb8HjgxeOT8h47'
        consumer_secret = '9dqRK0OVG5LAOTImEYMJC6N9fNEjolk16trsdiUaQpslr1Uv6E'
        access_token = '790131625287823360-1JOhf6LdjADGdojW1NRv6ywaHC9ifiL'
        access_token_secret = '16fkYIubpqihIMoSleEgf8SVB2QCgVgYK90XkreKNlk33'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

def biggest(a, y, z):
    Max = a
    if y > Max:
        Max = y
    if z > Max:
        Max = z
        if y > z:
            Max = y
    if Max == a: return 1
    if Max == y: return 0
    if Max == z: return -1
    
    

# creating object of TwitterClient Class
api = TwitterClient()
# calling function to get tweets
tweets = api.get_tweets(query = title, count = 10000)
 
# picking positive tweets from tweets
ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
# percentage of positive tweets
print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
# picking negative tweets from tweets
ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
# percentage of negative tweets
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
# percentage of neutral tweets
print("Neutral tweets percentage: {} % \
    ".format(100* (len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
    
nutweets = len(tweets) - len(ntweets) - len(ptweets)
    
Sentiments = biggest(len(ptweets),nutweets,len(ntweets))
print(Sentiments)


Positive tweets percentage: 45.56962025316456 %
Negative tweets percentage: 5.063291139240507 %
Neutral tweets percentage: 49.36708860759494 %     
0


In [159]:
data= []
data.append(title)

def popularity_fetch(url):
    try:
        res = requests.get(url)
        json_data = json.loads(res.text)
        return(json_data["results"][0]["popularity"])
    except requests.exceptions.RequestException as e:
        print(e)

def actor_popularity_fetch(url):
    try:
        res = requests.get(url)
        json_dat = json.loads(res.text)
        return(json_dat["results"][0]["popularity"])
    except requests.exceptions.RequestException as e:
        print(e)

movie = (re.sub(r'^[Tt]he',"", re.sub('[^\w\d-]',"", re.sub(r':.*',"",title))).lower() + ".htm")
url = 'http://www.boxofficemojo.com/movies/?id=' + str(movie)
print(url)
soupObjects={}  
try:
    resp = requests.get(url)
    soupObjects = BeautifulSoup(resp.text, 'lxml')
except requests.exceptions.RequestException as e:
    print(e)

pop_url = 'http://api.tmdb.org/3/search/person?api_key=d7721dd4346490aa61af275d888d2585&query='

# Distributor from bomojo
distributo = soupObjects.find(text=re.compile('Distributor:'))
if distributo:
    distributo = str(distributo.findNextSibling().text)
    data.append(distributo)

# director from bomojo
direc = soupObjects.find(text=re.compile('Director:'))
if direc:
    direct = direc.next_element()[0].text
    data.append(direct)
    d_op_url= pop_url + direc.next_element()[0].text
    director_popularity = popularity_fetch(d_op_url)
    data.append(director_popularity)

data.append(Sentiments)
# Actor from bomojo
act = soupObjects.find(text=re.compile('Actors:'))
if act:
    actor = act.next_element()[1].text
    data.append(actor)
    a_op_url= pop_url + act.next_element()[1].text
    actor_popularit = actor_popularity_fetch(a_op_url)
    data.append(actor_popularit)

# genre from bomojo
genr = soupObjects.find(text=re.compile('Genre:'))
if genr:
    genr = str(genr.findNextSibling().text)
    data.append(genr)

# Number of theatres from bomojo
num_of_theatres = soupObjects.find(text=re.compile('Widest Release:'))
if num_of_theatres:
    num_of_theatres = str(num_of_theatres.findNextSibling().text)
    data.append(num_of_theatres)

# runtime
runtime = soupObjects.find(text=re.compile('Runtime:'))
if runtime:
    run_time = str(runtime.findNextSibling().text)
    run_time = run_time.replace('N/A','0')
    runtime_mins = float(run_time.split(' ')[0])*60 + float(run_time.split(' ')[2])
    data.append(runtime_mins)

# production budget
pb = soupObjects.find(text=re.compile('Production Budget:'))    
if pb:
    prod_budget = str(pb.findNextSibling().text)
    prod_budget = prod_budget.replace('million','000000')
    prod_budget = prod_budget.replace('N/A|NA','0')
    prod_budget = prod_budget.replace(' ','')
    prod_budget = prod_budget.replace('$','')
    prod_budget = float(prod_budget)
    data.append(prod_budget)

# Release year from bomojo
release_year = soupObjects.find(text=re.compile('Release Date:'))
if release_year:
    release_year = str(release_year.findNextSibling().text)[-4:]
    data.append(int(release_year))

with open('predict.csv', 'a') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow([actor_popularit,director_popularity,Sentiments,500,runtime_mins,release_year,5.26E-11,cpi,prod_budget])
    
with open('predict1.csv', 'a') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow([genr,distributo,direct,actor])
    
    
dataset = pd.read_csv('predict.csv')
dataset1 = pd.read_csv('predict1.csv')
df = pd.DataFrame(dataset)
df1 = pd.DataFrame(dataset1)
cpi = 200
data.append(cpi)

print(data)

http://www.boxofficemojo.com/movies/?id=wonderwoman.htm
['Wonder Woman', 'Warner Bros.', 'Patty Jenkins', 1.068188, 0, 'Gal Gadot', 12.713087, 'Action / Adventure', 141.0, 149000000.0, 2017, 200]


In [160]:
df

,Actor popularity,Director popularity,Sentiment,num_theaters,runtime_mins,releaseYear,CULT_INDEX,CPI,prod_budget_ADJ
0,12.713087,1.068188,0,500,141.0,2017,5.260000e-11,200,149000000.0


In [161]:
df1

,genre,distributor,Director,leadActor
0,Action / Adventure,Warner Bros.,Patty Jenkins,Gal Gadot


In [162]:
g = df1.genre.str.get_dummies()

In [164]:
d = df1.distributor.str.get_dummies()

In [165]:
dr = df1.Director.str.get_dummies()

In [166]:
la = df1.leadActor.str.get_dummies()

In [169]:
df = pd.concat([df,d,g,dr,la], axis = 1)

In [170]:
df

,Actor popularity,Director popularity,Sentiment,num_theaters,runtime_mins,releaseYear,CULT_INDEX,CPI,prod_budget_ADJ,Warner Bros.,Action / Adventure,Patty Jenkins,Gal Gadot
0,12.713087,1.068188,0,500,141.0,2017,5.260000e-11,200,149000000.0,1,1,1,1


In [171]:
model.predict(df)

ValueError: shapes (1,13) and (6508,) not aligned: 13 (dim 1) != 6508 (dim 0)